In [26]:
import logging
import constants
import boto3
from botocore.exceptions import ClientError
import os

In [24]:
# use third party object storage to create an S3 Client
s3_client = boto3.client(
    "s3",
    endpoint_url=constants.AWS_S3_ENDPOINT,
    aws_access_key_id=constants.AWS_ACCESS_KEY_ID,
    aws_secret_access_key=constants.AWS_SECRET_ACCESS_KEY,
)

bucket = constants.AWS_S3_BUCKET

In [32]:
# specify the folder path (in the current working directory) for your object 
# if the folder doesn't exist, create it
save_path = os.path.join(os.getcwd(), r'junk')
if not os.path.exists(save_path):
    os.makedirs(save_path)

# create new text file within the folder
dest_file_path = os.path.join(save_path, r'test.txt')

# write data to the new text file
with open(dest_file_path, 'w') as fh:
    fh.write('example example example\n')

In [33]:
# put the new text file into your object storage bucket
s3_client.put_object(Bucket=bucket, Key=dest_file_path)

{'ResponseMetadata': {'RequestId': '8e22ee15:1893b9db5c9:263cba:2db2',
  'HostId': '496b7edd5f0b4c93460f7b8b259a0dd3599eff58bba18087a7e2fe8f88ef56ec',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 06 Feb 2024 17:12:10 GMT',
   'server': 'ViPR/1.0',
   'x-amz-request-id': '8e22ee15:1893b9db5c9:263cba:2db2',
   'x-amz-id-2': '496b7edd5f0b4c93460f7b8b259a0dd3599eff58bba18087a7e2fe8f88ef56ec',
   'etag': '"d41d8cd98f00b204e9800998ecf8427e"',
   'last-modified': 'Tue, 06 Feb 2024 17:12:10 GMT',
   'x-emc-mtime': '1707239530205',
   'x-emc-previous-object-size': '0',
   'x-amz-version-id': '1707239530205',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"d41d8cd98f00b204e9800998ecf8427e"',
 'VersionId': '1707239530205'}

In [34]:
# define the function for creating presigned URL 
# running the function will return a shareable URL in the print output
def create_presigned_url(
    bucket_name = bucket,
    object_name = dest_file_path,
    expiration=180,
):

    try:
        response = s3_client.generate_presigned_url(
            "get_object",
            Params={"Bucket": bucket_name, "Key": object_name},
            ExpiresIn=expiration,
            )
        print(response)

    except ClientError as e:
        logging.error(e)
        return None

    # The response contains the presigned URL
    return response

In [35]:
# run the presigned URL function with the given parameters
create_presigned_url(bucket, dest_file_path)

https://nrs.objectstore.gov.bc.ca:443/nrs-iit/c%3A%5CGit_Repo%5Cnr-optimize-obj-adm-api%5Cscripts%5Cjunk%5Ctest.txt?AWSAccessKeyId=nrs-iit-user&Signature=qFbdm6K6bCCMZawQt6kDbTJPnuc%3D&Expires=1707239722


'https://nrs.objectstore.gov.bc.ca:443/nrs-iit/c%3A%5CGit_Repo%5Cnr-optimize-obj-adm-api%5Cscripts%5Cjunk%5Ctest.txt?AWSAccessKeyId=nrs-iit-user&Signature=qFbdm6K6bCCMZawQt6kDbTJPnuc%3D&Expires=1707239722'

In [36]:
# delete the remote object
s3_client.delete_object(
    Bucket=bucket, 
    Key=dest_file_path
    )

{'ResponseMetadata': {'RequestId': '8e22ee15:1893b9db5c9:263333:6c54',
  'HostId': '496b7edd5f0b4c93460f7b8b259a0dd3599eff58bba18087a7e2fe8f88ef56ec',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'date': 'Tue, 06 Feb 2024 17:12:30 GMT',
   'server': 'ViPR/1.0',
   'x-amz-request-id': '8e22ee15:1893b9db5c9:263333:6c54',
   'x-amz-id-2': '496b7edd5f0b4c93460f7b8b259a0dd3599eff58bba18087a7e2fe8f88ef56ec',
   'x-amz-version-id': '1707239550301',
   'x-amz-delete-marker': 'true',
   'x-emc-previous-object-size': '0',
   'content-length': '0'},
  'RetryAttempts': 0},
 'DeleteMarker': True,
 'VersionId': '1707239550301'}